# KAKENの配分区分マスタをローカルのMariaDBに保存するプログラム

### 事前準備
- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、configparserで ./config.ini として保存しておく。

### ここから本編
bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [ ]:
# encoding: utf-8
from lxml import etree
import pandas as pd

In [ ]:
tree = etree.parse('grants_masterxml_kaken/section_master_kakenhi.xml')

sectionlist = []
for section_table in tree.iterfind("section_table"):
    for section in section_table.iterfind("section"):
        section_name = section.find("name[@lang='ja']").text
        section_niicode = section.find("code[@type='nii']").text

        row = [
            section_niicode,
            section_name,
        ]
        
        sectionlist.append(row)
        
df = pd.DataFrame(sectionlist)
df.columns = ['section_niicode', 'section_name']
df

重複のデータがあるかどうか

In [ ]:
df.duplicated().any()

重複データがあった。dropしておく。

In [ ]:
df = df.drop_duplicates()
df.duplicated().any()

In [ ]:
df.section_niicode.value_counts()

niicodeがユニークなので、インデックスに設定する

In [ ]:
df = df.set_index('section_niicode')
df

ローカルのmariaDBに関する設定ファイルを読み込み（config.iniはgitに上げていないが、事前準備で作られているはず）

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

データベースにテーブル構造とデータの中身を書き込む

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('kaken_master_section', engine, if_exists='replace',
          dtype={
              'section_niicode': Integer,
              'section_name': String(256),
                })

### おしまい
データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。